# Helm Demo Notebook

This notebook walks you through building a Docker image, deploying to Kubernetes, and packaging into a Helm chart with Istio VirtualService.

## Step 1: Build and Push Docker Image

In [4]:
import os

os.chdir(r"C:\Users\charbonv\Helm-demo\test\frontend\frontend")
print("Current directory:", os.getcwd())


Current directory: C:\Users\charbonv\Helm-demo\test\frontend\frontend


In [12]:
# create_voice2text_chart.py
import os
import shutil

chart_name = "voice2text"
chart_version = "0.1.1"
base_dir = f"./{chart_name}"
templates_dir = os.path.join(base_dir, "templates")

# Delete old chart if exists
if os.path.exists(base_dir):
    print(f"Removing existing chart at '{base_dir}'...")
    shutil.rmtree(base_dir)

# Create folder structure
print(f"Creating Helm chart directory structure at '{base_dir}'...")
os.makedirs(templates_dir, exist_ok=True)

# ---------------------------
# Chart.yaml
# ---------------------------
print("Writing Chart.yaml...")
chart_yaml = f"""\
apiVersion: v2
name: {chart_name}
description: Helm chart for vinchar/voice2text Gradio Meeting Capture app
type: application
version: {chart_version}
appVersion: "{chart_version}"
"""
with open(os.path.join(base_dir, "Chart.yaml"), "w") as f:
    f.write(chart_yaml)

# ---------------------------
# values.yaml
# ---------------------------
print("Writing values.yaml...")
values_yaml = """\
replicaCount: 1

image:
  repository: vinchar/voice2text
  tag: "1.2.4"
  pullPolicy: IfNotPresent

service:
  type: ClusterIP
  port: 7860

resources:
  limits: {}
  requests: {}

nodeSelector: {}

tolerations: []

affinity: {}

readinessProbe:
  path: "/"
  initialDelaySeconds: 5
  periodSeconds: 10
  timeoutSeconds: 5
  failureThreshold: 3
  successThreshold: 1

livenessProbe:
  path: "/"
  initialDelaySeconds: 30
  periodSeconds: 20
  timeoutSeconds: 5
  failureThreshold: 3
  successThreshold: 1

hpa:
  enabled: false
  minReplicas: 1
  maxReplicas: 3
  targetCPUUtilizationPercentage: 60

ingress:
  enabled: false
  # If using Kubernetes Ingress instead of Istio, configure host, annotations, tls etc.
  host: voice2text.example.com
  annotations: {}

# EzUA / Istio virtual service section (used by virtualservice.yaml)
ezua:
  virtualService:
    endpoint: "voice2text.${DOMAIN_NAME}"
    domain: "${DOMAIN_NAME}"
    istioGateway: "istio-system/ezaf-gateway"

# Optional persistence - not used by default but provided for extensibility
persistence:
  enabled: false
  storageClass: ""
  accessModes:
    - ReadWriteOnce
  size: 1Gi
"""
with open(os.path.join(base_dir, "values.yaml"), "w") as f:
    f.write(values_yaml)

# ---------------------------
# Deployment
# ---------------------------
print("Writing deployment.yaml...")
deployment_yaml = """\
apiVersion: apps/v1
kind: Deployment
metadata:
  name: {{ include "voice2text.fullname" . }}
  labels:
    {{- include "voice2text.labels" . | nindent 4 }}
spec:
  replicas: {{ .Values.replicaCount }}
  selector:
    matchLabels:
      app: {{ include "voice2text.name" . }}
  template:
    metadata:
      labels:
        app: {{ include "voice2text.name" . }}
    spec:
      {{- with .Values.nodeSelector }}
      nodeSelector:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      {{- with .Values.tolerations }}
      tolerations:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      {{- with .Values.affinity }}
      affinity:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      containers:
        - name: voice2text
          image: "{{ .Values.image.repository }}:{{ .Values.image.tag }}"
          imagePullPolicy: {{ .Values.image.pullPolicy }}
          ports:
            - containerPort: {{ .Values.service.port }}
          env:
            {{- range .Values.envSecrets }}
            - name: {{ .name }}
              valueFrom:
                secretKeyRef:
                  name: {{ .secretName }}
                  key: {{ .secretKey }}
            {{- end }}
          readinessProbe:
            httpGet:
              path: {{ .Values.readinessProbe.path | quote }}
              port: {{ .Values.service.port }}
            initialDelaySeconds: {{ .Values.readinessProbe.initialDelaySeconds }}
            periodSeconds: {{ .Values.readinessProbe.periodSeconds }}
            timeoutSeconds: {{ .Values.readinessProbe.timeoutSeconds }}
            failureThreshold: {{ .Values.readinessProbe.failureThreshold }}
            successThreshold: {{ .Values.readinessProbe.successThreshold }}
          livenessProbe:
            httpGet:
              path: {{ .Values.livenessProbe.path | quote }}
              port: {{ .Values.service.port }}
            initialDelaySeconds: {{ .Values.livenessProbe.initialDelaySeconds }}
            periodSeconds: {{ .Values.livenessProbe.periodSeconds }}
            timeoutSeconds: {{ .Values.livenessProbe.timeoutSeconds }}
            failureThreshold: {{ .Values.livenessProbe.failureThreshold }}
            successThreshold: {{ .Values.livenessProbe.successThreshold }}
          resources:
            {{- toYaml .Values.resources | nindent 12 }}
"""
with open(os.path.join(templates_dir, "deployment.yaml"), "w") as f:
    f.write(deployment_yaml)

# ---------------------------
# Service
# ---------------------------
print("Writing service.yaml...")
service_yaml = """\
apiVersion: v1
kind: Service
metadata:
  name: {{ include "voice2text.fullname" . }}
  labels:
    {{- include "voice2text.labels" . | nindent 4 }}
spec:
  type: {{ .Values.service.type }}
  ports:
    - port: {{ .Values.service.port }}
      targetPort: {{ .Values.service.port }}
      protocol: TCP
      name: http
  selector:
    app: {{ include "voice2text.name" . }}
"""
with open(os.path.join(templates_dir, "service.yaml"), "w") as f:
    f.write(service_yaml)

# ---------------------------
# HorizontalPodAutoscaler (optional)
# ---------------------------
print("Writing hpa.yaml...")
hpa_yaml = """\
{{- if .Values.hpa.enabled }}
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: {{ include "voice2text.fullname" . }}
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: {{ include "voice2text.fullname" . }}
  minReplicas: {{ .Values.hpa.minReplicas }}
  maxReplicas: {{ .Values.hpa.maxReplicas }}
  metrics:
    - type: Resource
      resource:
        name: cpu
        target:
          type: Utilization
          averageUtilization: {{ .Values.hpa.targetCPUUtilizationPercentage }}
{{- end }}
"""
with open(os.path.join(templates_dir, "hpa.yaml"), "w") as f:
    f.write(hpa_yaml)

# ---------------------------
# VirtualService (Istio)
# ---------------------------
print("Writing virtualservice.yaml...")
virtualservice_yaml = """\
apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: {{ include "voice2text.fullname" . }}
  namespace: {{ .Release.Namespace }}
  labels:
    {{- include "voice2text.labels" . | nindent 4 }}
spec:
  gateways:
    - {{ .Values.ezua.virtualService.istioGateway }}
  hosts:
    - {{ .Values.ezua.virtualService.endpoint }}
  http:
    - match:
        - uri:
            prefix: /
      rewrite:
        uri: /
      route:
        - destination:
            host: {{ include "voice2text.fullname" . }}.{{ .Release.Namespace }}.svc.cluster.local
            port:
              number: {{ .Values.service.port }}
"""
with open(os.path.join(templates_dir, "virtualservice.yaml"), "w") as f:
    f.write(virtualservice_yaml)

# ---------------------------
# NOTES.txt
# ---------------------------
print("Writing NOTES.txt...")
notes_txt = """\
Thank you for installing the voice2text chart!

To create Kubernetes secrets for HF & OpenAI tokens, run (example):
  kubectl create secret generic hf-secret --from-literal=hf_api_token='<YOUR_HF_TOKEN>' -n <namespace>
  kubectl create secret generic openai-secret --from-literal=openai_api_key='<YOUR_OPENAI_KEY>' -n <namespace>

Install the chart:
  helm install voice2text ./voice2text

To upgrade with new values:
  helm upgrade --install voice2text ./voice2text -f my-values.yaml

If using Istio, ensure your gateway in values (ezua.virtualService.istioGateway) is correct.

By default the service listens on port 7860 (Gradio). Use port-forward for local testing:
  kubectl port-forward svc/$(kubectl get svc -l app.kubernetes.io/name=voice2text -o jsonpath='{.items[0].metadata.name}') 7860:7860

"""
with open(os.path.join(templates_dir, "NOTES.txt"), "w") as f:
    f.write(notes_txt)

# ---------------------------
# Helpers (names, labels)
# ---------------------------
print("Writing _helpers.tpl...")
helpers_tpl = """\
{{/*
Chart name (voice2text)
*/}}
{{- define "voice2text.name" -}}
{{- .Chart.Name | trunc 63 | trimSuffix "-" -}}
{{- end }}

{{/*
Full name (release-voice2text)
*/}}
{{- define "voice2text.fullname" -}}
{{- printf "%s-%s" .Release.Name (include "voice2text.name" .) | trunc 63 | trimSuffix "-" -}}
{{- end }}

{{/*
Standard labels
*/}}
{{- define "voice2text.labels" -}}
app.kubernetes.io/name: {{ include "voice2text.name" . }}
app.kubernetes.io/instance: {{ .Release.Name }}
app.kubernetes.io/version: {{ .Chart.AppVersion }}
app.kubernetes.io/managed-by: {{ .Release.Service }}
{{- end }}
"""
with open(os.path.join(templates_dir, "_helpers.tpl"), "w") as f:
    f.write(helpers_tpl)

print(f"Helm chart folder created at: {base_dir}")

# ---------------------------
# Package the chart
# ---------------------------
print("Packaging Helm chart into .tgz archive...")
exit_code = os.system(f"helm package {base_dir}")

if exit_code == 0:
    print("Chart packaged successfully!")
    print(f"Install with:")
    print(f"  helm install voice2text {chart_name}-{chart_version}.tgz")
else:
    print("Failed to package chart. Ensure Helm is installed and on PATH.")


Removing existing chart at './voice2text'...
Creating Helm chart directory structure at './voice2text'...
Writing Chart.yaml...
Writing values.yaml...
Writing deployment.yaml...
Writing service.yaml...
Writing hpa.yaml...
Writing virtualservice.yaml...
Writing NOTES.txt...
Writing _helpers.tpl...
Helm chart folder created at: ./voice2text
Packaging Helm chart into .tgz archive...
Chart packaged successfully!
Install with:
  helm install voice2text voice2text-0.1.1.tgz
